# Instalación:


### Las primeras 3 celdas solo se ejecutan 1 vez




In [1]:
import os, sys
from google.colab import drive

drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileExistsError: ignored

## Instalación de biblioteca Stable Baseline



### Biblioteca que contiene los modelos


In [ ]:
!pip install --target=$nb_path git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

  Cloning https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to /tmp/pip-req-build-hl240wcw
  Running command git clone --filter=blob:none --quiet https://github.com/Stable-Baselines-Team/stable-baselines3-contrib /tmp/pip-req-build-hl240wcw
  Resolved https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to commit 707cb0f9cd9104c989a3e4d4cf30965b64ca7339
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached stable_baselines3-2.2.1-py3-none-any.whl (181 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for sb3-contrib: filename=sb3_contrib-2.2.1-py3-none-any.whl size=80668 sha256=99c650767f99181aebf0b177060706bc9e87222b80a032605d3c5213ded7341d
  Stored in directory: /tmp/pip-ephem-wheel-cache-gqfi9_1t/wheels/f8/e1/97/7e886248901c65ffe4ff7a9c16bccfbde992692ac10d99ee41
S

## Instalación del Dream On Gym

### Contiene el ambiente de redes opticas y el simulador

In [ ]:
!pip install --target=$nb_path dream-on-gym-v2==0.0.6

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 53.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.8 MB/s eta 0:00:00
  Created wheel for dream-on-gym-v2: filename=dream_on_gym_v2-0.0.6-py3-none-any.whl size=29791 sha256=c7908c42803a8c55f7d0a8



---

# Codigo



In [2]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch as th

from sb3_contrib import TRPO
from sb3_contrib.trpo import MlpPolicy as MLP_TRPO

from dreamongymv2.simNetPy import *
from dreamongymv2.gym_basic import *
import gymnasium

sys.modules["gym"] = gymnasium

In [4]:
bands = ['L','C','S','E']

episodeRewardAccum = 0
iteration = 0
episodeIteration = 100
episodeRewardsArray = []

In [5]:
def getBand(actionBand):
    if (actionBand == 0):
        return 'C'
    elif (actionBand == 1):
        return 'L'
    elif (actionBand == 2):
        return 'S'
    elif (actionBand == 3):
        return 'E'

    return 'NoBand'

In [6]:
def reward():
    global episodeRewardAccum
    global episodeIteration
    global episodeRewardsArray
    global iteration

    iteration = iteration + 1
    if (iteration % episodeIteration) == 0:
        episodeRewardsArray.append(episodeRewardAccum)
        episodeRewardAccum = 0

    value = env.getSimulator().lastConnectionIsAllocated()
    if (value.name == Controller.Status.Not_Allocated.name):
        #print(value.name)
        value = -1
    else:
        value = 1
    episodeRewardAccum = episodeRewardAccum + value
    return value

In [7]:
def state():
    connectionEvent = env.getSimulator().connectionEvent
    route = env.getSimulator().controller.path[connectionEvent.source][connectionEvent.destination][0]
    slotsQuantity = 0
    for i in range(4):
        slotsQuantity = slotsQuantity + route[0].getSlots(getBand(i))
    obs = [0] * slotsQuantity
    slotInit = 0
    for i in range(len(route)):
        slotIterator = 0
        for j in range(4):
            slotInit = slotIterator
            for k in range(slotInit, slotInit + route[i].getSlots(getBand(j))):
                slotIterator = slotIterator + 1
                if route[i].getSlot(k-slotInit,getBand(j)):
                    obs[k] = obs[k] + 1
    return obs


In [8]:
def first_fit_algorithm(src: int, dst: int, b: BitRate, c: Connection, n: Network, path, action):
    numberOfSlots = b.getNumberofSlots(0)
    link_ids = path[src][dst][0]
    c.bandSelected = "C"
    general_link = []
    for _ in range(n.getLink(0).getSlots(c.bandSelected)):
        general_link.append(False)

    for link in link_ids:
        link = n.getLink(link.id)
        link.bandSelected = c.bandSelected
        for slot in range(link.getSlots(c.bandSelected)):
            general_link[slot] = general_link[slot] or link.getSlot(slot,c.bandSelected)
    currentNumberSlots = 0
    currentSlotIndex = 0
    for j in range(len(general_link)):
        if not general_link[j]:
            currentNumberSlots += 1
        else:
            currentNumberSlots = 0
            currentSlotIndex = j + 1
        if currentNumberSlots == numberOfSlots:
            for k in link_ids:
                c.addLink(
                    k, fromSlot=currentSlotIndex, toSlot=currentSlotIndex+currentNumberSlots)
            return Controller.Status.Allocated, c
    return Controller.Status.Not_Allocated, c

In [9]:
def agent_algorithm(src: int, dst: int, b: BitRate, c: Connection, n: Network, path, action):
    band = getBand(action[0])
    c.bandSelected = band
    numberOfSlots = b.getNumberofSlots(0)
    actionSpace = len(path[src][dst])
    link_ids = path[src][dst][0]
    general_link = []
    for _ in range(n.getLink(0).getSlots(band)):
        general_link.append(False)
    for link in link_ids:
        link = n.getLink(link.id)
        link.bandSelected = band
        for slot in range(link.getSlots()):
            general_link[slot] = general_link[slot] or link.getSlot(slot,band)
    currentNumberSlots = 0
    currentSlotIndex = 0
    for j in range(len(general_link)):
        if not general_link[j]:
            currentNumberSlots += 1
        else:
            currentNumberSlots = 0
            currentSlotIndex = j + 1
        if currentNumberSlots == numberOfSlots:
            for k in link_ids:
                c.addLink(
                    k, fromSlot=currentSlotIndex, toSlot=currentSlotIndex+currentNumberSlots)
            return Controller.Status.Allocated, c
    return Controller.Status.Not_Allocated, c

In [10]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(reward)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/BDM_ARPANet_4_bands.json", fileDirectory + "/Topologias-rutas/ARPANet_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

env.start()
env.getSimulator().setAllocator(agent_algorithm)
policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setRewardFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setRewardFunc` for environment variables or `env.get_wrapper_attr('setRewardFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setStateFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setStateFunc` for environment variables or `env.get_wrapper_attr('setStateFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.initEnviroment to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.initEnviroment` for environment variables

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01
Nodes: 20
Links: 78
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+
|     5.0% |     5000 | 0.322735 |  0:00:05 | 0.013222 | 0.013218 | 0.013218 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.332867 |  0:00:08 | 0.009424 | 0.009423 | 0.009423 |
+----------+----------+----------+----------+----------+----------+----------+


KeyboardInterrupt: ignored

# Instalcion de Optuna y Optuna-Dashboard

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.2 MB/s eta 0:00:00


In [ ]:
!pip install optuna-dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 9.2 MB/s eta 0:00:00


## Optimizacion con Optuna

In [ ]:
import optuna
import shutil

def objective(trial):
    global episodeRewardAccum
    global episodeIteration
    global episodeRewardsArray
    global iteration
    global env

    bands = ['L','C','S','E']

    episodeRewardAccum = 0
    iteration = 0
    episodeIteration = 100
    episodeRewardsArray = []

    fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

    env = gymnasium.make("rlonenv-v0")

    obs, info = env.reset()

    #definir espacio (opciones del 0-3, 4 en total )
    env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


    #tamaño de los slots de las bandas
    env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

    #Set funcion recompensa
    env.setRewardFunc(reward)
    #Set estados
    env.setStateFunc(state)
    #env.setStateFunc(stateAllNetwork)
    #Definir topologia y rutas
    env.initEnviroment(fileDirectory + "/Topologias-rutas/BDM_ARPANet_4_bands.json", fileDirectory + "/Topologias-rutas/ARPANet_routes.json")
    #Cantidad de conexiones para tener el simulador corriendo
    env.getSimulator().goalConnections = 100000
    env.getSimulator().setMu(1)
    env.getSimulator().setLambda(1000)
    env.getSimulator().setAllocator(first_fit_algorithm)
    env.getSimulator().init()

    env.start()
    env.getSimulator().setAllocator(agent_algorithm)


    print("Se reseteo!!!!!!!!", len(episodeRewardsArray))


    # Define hiperparámetros a optimizar
    net_arch = [trial.suggest_int('net_arch_' + str(i), 32, 256) for i in range(5)]  ### REVISAR SI SON 5 CAPAS XD
    gamma = trial.suggest_float('gamma', 0.9, 0.99)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 32, 512)

    #model = TRPO(MLP_TRPO, env, verbose=0, tensorboard_log="./tb/TRPO-DeepRBMLSA-v0/", policy_kwargs=policy_args, gamma=.95)

    # Crea el modelo con los hiperparámetros
    model = TRPO(MLP_TRPO, env, verbose=0, tensorboard_log="./tb/TRPO-DeepRBMLSA-v0/",
                 policy_kwargs=dict(net_arch=net_arch, activation_fn=th.nn.ReLU), gamma=gamma, learning_rate=learning_rate, batch_size=batch_size)

    # Entrenar el modelo
    model.learn(total_timesteps=1000, log_interval=4)

    print("Post train", len(episodeRewardsArray), episodeRewardsArray)

    trial.set_user_attr('episode_rewards', len(episodeRewardsArray))
    # trial.set_user_attr('rewards', np.mean(rewardsArray))

    return np.mean(episodeRewardsArray)

# Crea un estudio Optuna
study = optuna.create_study(direction='maximize', storage='sqlite:///example.db')

# Realiza la optimización de hiperparámetros
study.optimize(objective, n_trials=30)

print('Best trial:')
best_trial = study.best_trial

print('Value: ', best_trial.value)
print('Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')

## GUARDAR BASE DE DATOS

In [ ]:
import shutil

# Ruta del archivo de base de datos SQLite en Colab
source_path = 'example.db'  # Reemplaza con la ruta correcta

# Ruta en Google Drive
drive_path = '/content/drive/MyDrive/outputs/'

# Copiar la base de datos a Google Drive
shutil.copy(source_path, drive_path)

'/content/drive/MyDrive/outputs/example.db'

## GUARDAR MODELO

In [ ]:
best_trial = study.best_trial

parametros_optimizados = best_trial.params.items()
parametros_optimizados = dict(parametros_optimizados)
# Extraer los parámetros específicos
net_arch = [parametros_optimizados[f'net_arch_{i}'] for i in range(5)]
gamma = parametros_optimizados['gamma']
learning_rate = parametros_optimizados['learning_rate']
batch_size = parametros_optimizados['batch_size']
model = TRPO(MLP_TRPO, env, verbose=0, tensorboard_log="./tb/TRPO-DeepRBMLSA-v0/",
                policy_kwargs=dict(net_arch=net_arch, activation_fn=th.nn.ReLU), gamma=gamma, learning_rate=learning_rate, batch_size=batch_size)

model.learn(total_timesteps=1000, log_interval=4)
modelDirectory = fileDirectory + "/models/"
model.save(modelDirectory+ "TRPO")

## CONEXION CON OPTUNA DASHBOARD

In [11]:
#Se carga el study desde la bd
import time
import threading
from optuna_dashboard import wsgi
import optuna
from wsgiref.simple_server import make_server

port = 4439

# Especifica la ruta completa a la base de datos SQLite
database_path = "/content/drive/MyDrive/outputs/example.db"

# Configura la conexión a la base de datos SQLite
storage = optuna.storages.RDBStorage(
    url=f"sqlite:///{database_path}",
    engine_kwargs={"connect_args": {"check_same_thread": False}},
)

# Ahora puedes utilizar 'storage' en Optuna

s_name = "no-name-4bb6b584-bff7-4818-9bf0-2aef5152f472"

study = optuna.load_study(study_name=s_name, storage=storage)
#study = optuna.create_study(storage=storage)

In [ ]:
#optuna dashboard al localhost
app = wsgi(storage)
httpd = make_server("localhost", port, app)
thread = threading.Thread(target=httpd.serve_forever)
thread.start()
time.sleep(3) # Wait until the server startup

from google.colab import output
output.serve_kernel_port_as_window(port, path='/dashboard/')

# CARGA Y TEST MODELO

In [12]:
fileDirectory = '/content/drive/MyDrive/DREAMONGYM'

env = gymnasium.make("rlonenv-v0")

obs, info = env.reset()

#definir espacio (opciones del 0-3, 4 en total )
env.action_space = gymnasium.spaces.MultiDiscrete(np.array([4]))


#tamaño de los slots de las bandas
env.observation_space = gymnasium.spaces.MultiDiscrete([100] * 2720)

#Set funcion recompensa
env.setRewardFunc(reward)
#Set estados
env.setStateFunc(state)
#env.setStateFunc(stateAllNetwork)
#Definir topologia y rutas
env.initEnviroment(fileDirectory + "/Topologias-rutas/NSFNet_4_bands.json", fileDirectory + "/Topologias-rutas/NSFNet_routes.json")
#Cantidad de conexiones para tener el simulador corriendo
env.getSimulator().goalConnections = 100000
env.getSimulator().setMu(1)
env.getSimulator().setLambda(1000)
env.getSimulator().setAllocator(first_fit_algorithm)
env.getSimulator().init()

env.start()
env.getSimulator().setAllocator(agent_algorithm)
policy_args = dict(net_arch=5*[128], activation_fn=th.nn.ReLU)

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setRewardFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setRewardFunc` for environment variables or `env.get_wrapper_attr('setRewardFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.setStateFunc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.setStateFunc` for environment variables or `env.get_wrapper_attr('setStateFunc')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.initEnviroment to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.initEnviroment` for environment variables

meta: 0.95
objetivo: 0.95 covered: 0.0 epsilon: 0.01
meta: 0.2673105078629141
objetivo: 0.95 covered: 0.6826894921370859 epsilon: 0.01
Nodes: 14
Links: 44
Goal Connections: 100000
Lambda: 1000
Mu: 1
+----------+----------+----------+----------+----------+----------+----------+
| progress |  arrives | blocking |  time(s) |  Wald CI |  A-C. CI |Wilson CI |
+----------+----------+----------+----------+----------+----------+----------+
|     5.0% |     5000 | 0.418316 |  0:00:02 | 0.013951 | 0.013945 | 0.013945 |
+----------+----------+----------+----------+----------+----------+----------+
|    10.0% |    10000 | 0.425957 |  0:00:05 | 0.009889 | 0.009887 | 0.009887 |
+----------+----------+----------+----------+----------+----------+----------+
|    15.0% |    15000 | 0.426772 |  0:00:07 | 0.008077 | 0.008076 | 0.008076 |
+----------+----------+----------+----------+----------+----------+----------+
|    20.0% |    20000 | 0.424179 |  0:00:11 | 0.006989 | 0.006988 | 0.006988 |
+----------

In [13]:
import torch
from sb3_contrib import TRPO
from sb3_contrib.trpo import MlpPolicy as MLP_TRPO

best_trial = study.best_trial

parametros_optimizados = best_trial.params.items()
parametros_optimizados = dict(parametros_optimizados)
# Extraer los parámetros específicos
net_arch = [parametros_optimizados[f'net_arch_{i}'] for i in range(5)]
gamma = parametros_optimizados['gamma']
learning_rate = parametros_optimizados['learning_rate']
batch_size = parametros_optimizados['batch_size']
model = TRPO(MLP_TRPO, env, verbose=0, tensorboard_log="./tb/TRPO-DeepRBMLSA-v0/",
                policy_kwargs=dict(net_arch=net_arch, activation_fn=th.nn.ReLU), gamma=gamma, learning_rate=learning_rate, batch_size=batch_size)

model.load("/content/drive/MyDrive/DREAMONGYM/models/TRPO.zip")

/usr/local/lib/python3.10/dist-packages/sb3_contrib/trpo/trpo.py:154: UserWarning: You have specified a mini-batch size of 423, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2048`, after every 4 untruncated mini-batches, there will be a truncated mini-batch of size 356
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2048 and n_envs=1)
  warnings.warn(


In [ ]:
obs, info = env.reset()
rewardsArray = []
rewardTemp = 0
iterationAccum = 100
_state = [0] * 2720
for i in range(100000):
    if (i % iterationAccum) == 0:
        rewardsArray.append(rewardTemp)
        rewardTemp = 0
    action, _states = model.predict(_state)
    _state, _reward, terminated, truncated, info = env.step(action)
    rewardTemp = rewardTemp + _reward
print(episodeRewardsArray)
print(rewardsArray, np.mean(rewardsArray[1:]))

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.getSimulator to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.getSimulator` for environment variables or `env.get_wrapper_attr('getSimulator')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'list'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir "./tb/TRPO-DeepRBMLSA-v0/"